In [100]:
# 0 azimuth is North
# 360 = 0
spt = 2**14
tpr = 5
degree_per_turn = 360/tpr
degree_per_step = degree_per_turn/spt
turn_per_degree = tpr/360
home_az = 90
home_steps = 10000
home_turns = 2
home_rotpos = get_rotpos(home_steps, home_turns)
north_rotpos = home_rotpos + ((0 - home_az)/degree_per_turn)
assert degree_per_step*spt == degree_per_turn
print(f"{degree_per_step=}\n{degree_per_turn=}\n{turn_per_degree=}\n{home_rotpos=}\n{north_rotpos=}")

get_rotpos: steps/spt=0.6103515625, turns=2
degree_per_step=0.00439453125
degree_per_turn=72.0
turn_per_degree=0.013888888888888888
home_rotpos=2.6103515625
north_rotpos=1.3603515625


In [104]:
def get_rotpos(steps, turns):
    print(f"get_rotpos: {steps/spt=}, {turns=}")
    return steps/spt + turns
    
def steps_turn_to_az(steps, turns):
    rotpos = get_rotpos(steps, turns)
    print(f"steps_turn_to_az: {rotpos=}, {rotpos-north_rotpos=}")
    return ((rotpos - north_rotpos)*degree_per_turn) % 360
    

In [105]:
print("should be 90:", steps_turn_to_az(10000, 2))

get_rotpos: steps/spt=0.6103515625, turns=2
steps_turn_to_az: rotpos=2.6103515625, rotpos-north_rotpos=1.25
should be 90: 90.0


In [107]:
steps_turn_to_az(10000, 7)

get_rotpos: steps/spt=0.6103515625, turns=7
steps_turn_to_az: rotpos=7.6103515625, rotpos-north_rotpos=6.25


90.0

In [155]:
RIGHT = 1
LEFT = 2

def direction_finder(current_az, target_az):
    diff = abs(target_az - current_az)
    is_left = (target_az - current_az) < 0
    return is_left, diff
    #back_dir = abs(current_az + (360-target_az))
    #return forward_dir, back_dir

def new_direction_finder(current_az, target_az):
    direction = 0
    diff = abs(current_az - target_az)
    diff2 = ((diff % 360) + 360) % 360;
    if diff == 0:
         direction = 0
    if diff < 180:
        # Rotate current directly towards target.
        direction = RIGHT if current_az < target_az else LEFT;
    else:
        # Rotate the other direction towards target.
        direction = LEFT if current_az < target_az else RIGHT;
    print(f"Currentaz = {current_az}, {target_az=}")
    return direction, diff, diff2
    

In [156]:

t1=(90,100)
r1=(RIGHT)
t2=(90,350)
r2=(LEFT)
t3=(350,10)
r3=(RIGHT)
t4=(359,0)
r4=(RIGHT)
t5=(0,180)
r5=(RIGHT)
t6=(10,359)
r6=(LEFT)


tests = [t1, t2, t3, t4, t5, t6]
results = [r1, r2, r3, r4, r5, r6]

for idx, test in enumerate(tests):
    direction, diff, diff2 = new_direction_finder(*test)
    print(f"{test=}, {res=}, {direction == results[idx]}, {diff=}, {diff2=}")
    



Currentaz = 90, target_az=100
test=(90, 100), res=(False, 180), True, diff=10, diff2=10
Currentaz = 90, target_az=350
test=(90, 350), res=(False, 180), True, diff=260, diff2=260
Currentaz = 350, target_az=10
test=(350, 10), res=(False, 180), True, diff=340, diff2=340
Currentaz = 359, target_az=0
test=(359, 0), res=(False, 180), True, diff=359, diff2=359
Currentaz = 0, target_az=180
test=(0, 180), res=(False, 180), False, diff=180, diff2=180
Currentaz = 10, target_az=359
test=(10, 359), res=(False, 180), True, diff=349, diff2=349
